## Azure SQL tips: USE and GO

- GO isn’t T‑SQL; it’s a client tool batch separator used by SSMS/sqlcmd. Don’t include `GO` in SQL sent via drivers (pyodbc); split batches as separate commands.
- USE to change database context isn’t appropriate in typical Azure SQL Database application queries. Connect directly to the desired database. For cross‑database access, consider Elastic query or Azure SQL Managed Instance.
- In this notebook, we avoid multi‑batch scripts and run single executable statements; we also skip execution for empty or non‑SELECT outputs to prevent driver errors.

# NL→SQL Demo Notebook (Azure OpenAI + Azure SQL)

This notebook walks through the NL→Intent→SQL pipeline step by step (aligned with the project flowchart).
Each code cell is paired with guidance to help you present or explore the pipeline (for TERADATA demo).

## Demo settings (global)

- Control printing and execution across the notebook from one place.
- You can still override the query text in runner cells if needed.


In [1]:
# Imports and environment setup
import os, re, sys, json, time
from typing import List, Dict, Any
from datetime import datetime
from dotenv import load_dotenv

# Third-party
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Load .env (expects Azure OpenAI + Azure SQL vars)
load_dotenv()
print('Environment loaded. Ready to configure Azure OpenAI and Azure SQL.')

Environment loaded. Ready to configure Azure OpenAI and Azure SQL.


In [ ]:
# Demo settings (global)
# These control printing and execution across the notebook.
SHOW_INTENT = True
SHOW_REASONING = True
NO_EXEC = True
REFRESH_SCHEMA = False

# Default query used by runner cells unless overridden
default_test_query = 'Weighted average interest rate by region'
print('Demo settings -> SHOW_INTENT:', SHOW_INTENT, '| SHOW_REASONING:', SHOW_REASONING, '| NO_EXEC:', NO_EXEC, '| REFRESH_SCHEMA:', REFRESH_SCHEMA)

## Azure OpenAI configuration & reasoning models note
- o-series/GPT-5 reasoning models using Chat Completions do not support temperature/top_p, etc.
- We detect such deployments and call the REST API directly, omitting unsupported params.

In [2]:
# Azure OpenAI configuration
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2025-04-01-preview')

def _is_reasoning_like_model(deployment_name: str | None) -> bool:
    name = (deployment_name or '').lower()
    return name.startswith('o') or name.startswith('gpt-5')

_USING_REASONING_STYLE = _is_reasoning_like_model(DEPLOYMENT_NAME)
print('Using reasoning style:', _USING_REASONING_STYLE, '| Deployment:', DEPLOYMENT_NAME)

# Direct Chat Completions for reasoning models (no temperature/top_p)
import requests

def _azure_chat_completions(messages: List[Dict[str, Any]], max_completion_tokens: int | None = None) -> str:
    url = f"{ENDPOINT.rstrip('/')}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    payload: Dict[str, Any] = {'messages': messages}
    if max_completion_tokens is not None:
        payload['max_completion_tokens'] = max_completion_tokens
    headers = {'api-key': API_KEY or '', 'Content-Type': 'application/json'}
    resp = requests.post(url, headers=headers, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    return data['choices'][0]['message']['content']

# LangChain wrapper for non-reasoning models

def _make_llm():
    if _USING_REASONING_STYLE:
        return None
    return AzureChatOpenAI(
        openai_api_key=API_KEY,
        azure_endpoint=ENDPOINT,
        deployment_name=DEPLOYMENT_NAME,
        api_version=API_VERSION,
        max_tokens=8192,
    )

llm = _make_llm()
print('LLM configured (None for reasoning models):', llm is None)

Using reasoning style: True | Deployment: gpt-5
LLM configured (None for reasoning models): True


## Prompts and intent extraction (diagram: INTENT & ENTITIES)
We maintain shared prompt text and pick the correct call-path (direct REST vs LangChain).

In [3]:
# Prompt text
INTENT_PROMPT_TEXT = (
    """
    You are an expert in translating natural language to database queries. Extract the intent and entities from the following user input:
    {input}
    """
)
SQL_PROMPT_TEXT = (
    """
    You are an expert in SQL and Azure SQL Database. Given the following database schema and the intent/entities, generate a valid T-SQL query for querying the database.

    IMPORTANT:
    - Do NOT use USE statements (USE [database] is not supported in Azure SQL Database)
    - Generate only the SELECT query without USE or GO statements
    - Return only executable T-SQL code without markdown formatting
    - The database connection is already established to the correct database

    Schema:
    {schema}
    Intent and Entities:
    {intent_entities}

    Generate a T-SQL query that can be executed directly:
    """
)
REASONING_PROMPT_TEXT = (
    """
    You are assisting with SQL generation. Before writing SQL, produce a short, high-level reasoning summary
    for how you will construct the query, based on the schema and the intent/entities.

    Rules:
    - Do NOT include any SQL code.
    - Keep it concise (<= 150 words) and actionable.
    - Use simple bullets covering: Entities mapping, Tables/Joins, Aggregations (if any), Filters, Order/Limit, Assumptions.

    Schema:
    {schema}
    Intent and Entities:
    {intent_entities}

    Provide the reasoning summary now:
    """
)

# Templates for LangChain path
intent_prompt = ChatPromptTemplate.from_template(INTENT_PROMPT_TEXT)
sql_prompt = ChatPromptTemplate.from_template(SQL_PROMPT_TEXT)
reasoning_prompt = ChatPromptTemplate.from_template(REASONING_PROMPT_TEXT)

def parse_nl_query(user_input: str) -> str:
    if _USING_REASONING_STYLE:
        prompt_text = INTENT_PROMPT_TEXT.format(input=user_input)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = intent_prompt | llm
    res = chain.invoke({'input': user_input})
    return res.content

## Schema context & optional refresh (diagram: REFRESH SCHEMA branch)
The schema context steers SQL generation to real tables/columns. Optionally refresh the cache.

In [ ]:
# Safe imports from this repository (resolve repo root even if running from docs/ or elsewhere)
from pathlib import Path

def _add_repo_root_to_sys_path(target_files=("schema_reader.py",)) -> Path | None:
    cwd = Path(os.getcwd()).resolve()
    # Walk up up to 5 levels to find a dir containing any of target_files
    for parent in [cwd] + list(cwd.parents)[:5]:
        for fname in target_files:
            if (parent / fname).exists():
                parent_str = str(parent)
                if parent_str not in sys.path:
                    sys.path.insert(0, parent_str)
                return parent
    return None

def _safe_import_schema_reader():
    _add_repo_root_to_sys_path(("schema_reader.py", "nl2sql_main.py"))
    try:
        from schema_reader import get_sql_database_schema_context  # type: ignore
        return get_sql_database_schema_context
    except Exception as e:
        raise ImportError('Unable to import get_sql_database_schema_context from schema_reader.') from e

# Use global REFRESH_SCHEMA from the Demo settings cell
if REFRESH_SCHEMA:
    try:
        _add_repo_root_to_sys_path(("schema_reader.py",))
        import schema_reader  # type: ignore
        cache_path = schema_reader.refresh_schema_cache()
        print('[INFO] Schema cache refreshed:', cache_path)
    except Exception as e:
        print('[WARN] Failed to refresh schema cache:', e)

## Reasoning summary (optional)
Ask the model for a short plan before generating SQL (skippable in faster runs).

In [ ]:
def generate_reasoning(intent_entities: str) -> str:
    get_schema_ctx = _safe_import_schema_reader()
    schema = get_schema_ctx()
    if _USING_REASONING_STYLE:
        prompt_text = REASONING_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = reasoning_prompt | llm
    res = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return res.content

# Use default_test_query unless overridden above
user_query = default_test_query
intent_entities = parse_nl_query(user_query)
if SHOW_INTENT:
    print('INTENT & ENTITIES:\n', intent_entities, '\n')

if SHOW_REASONING:
    try:
        reasoning = generate_reasoning(intent_entities)
        print('SQL GENERATION REASONING:\n', reasoning, '\n')
    except Exception as e:
        print('[WARN] Reasoning step failed:', e)

INTENT & ENTITIES:
 {
  "intent": "list_loans",
  "entities": {
    "resource": "loans",
    "limit": 10,
    "sort": {
      "by": "recency",
      "order": "desc"
    }
  }
} 

SQL GENERATION REASONING:
 - Entities mapping: “loans” → dbo.vw_LoanPortfolio rows (one per loan).
- Tables/Joins: Use dbo.vw_LoanPortfolio only; no joins needed for a simple list.
- Columns: Return key loan fields (LoanId, LoanNumber, CompanyName, PrincipalAmount, CurrencyCode, OriginationDate, MaturityDate, InterestRatePct, InterestRateType, Status).
- Aggregations: None (no grouping needed).
- Filters: None specified; include all loans.
- Order/Limit: Sort by recency → assume OriginationDate DESC; add LoanId DESC as tiebreaker. Limit to 10.
- Assumptions: “Recency” refers to most recently originated loans (OriginationDate). No status filter (e.g., active only) unless specified. No currency conversion required. 



## SQL generation (diagram: SQL GENERATION)
Produce a T-SQL statement aligned with the extracted intent and schema context.

In [6]:
def generate_sql(intent_entities: str) -> str:
    get_schema_ctx = _safe_import_schema_reader()
    schema = get_schema_ctx()
    if _USING_REASONING_STYLE:
        prompt_text = SQL_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = sql_prompt | llm
    result = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return result.content

raw_sql = generate_sql(intent_entities)
print('GENERATED SQL (RAW):\n', raw_sql, '\n')

GENERATED SQL (RAW):
 SELECT TOP (10)
    LoanId,
    LoanNumber,
    CompanyName,
    Industry,
    CreditRating,
    CountryName,
    RegionName,
    OriginationDate,
    MaturityDate,
    PrincipalAmount,
    CurrencyCode,
    InterestRatePct,
    InterestRateType,
    ReferenceRate,
    SpreadBps,
    AmortizationType,
    PaymentFreqMonths,
    Status,
    Purpose
FROM dbo.vw_LoanPortfolio
ORDER BY OriginationDate DESC, LoanId DESC 



## SQL sanitization (diagram: SANITIZATION)
Extract the SELECT code and normalize quotes for execution.

In [7]:
def extract_and_sanitize_sql(raw_sql: str) -> str:
    sql_code = raw_sql
    code_block = re.search(r"```sql\s*([\s\S]+?)```", raw_sql, re.IGNORECASE)
    if not code_block:
        code_block = re.search(r"```([\s\S]+?)```", raw_sql)
    if code_block:
        sql_code = code_block.group(1).strip()
    else:
        select_match = re.search(r"(SELECT[\s\S]+)", raw_sql, re.IGNORECASE)
        if select_match:
            sql_code = select_match.group(1).strip()
    return (
        sql_code.replace('’', "'")
                .replace('‘', "'")
                .replace('“', '"')
                .replace('”', '"')
    )

sql_to_run = extract_and_sanitize_sql(raw_sql)
print('SANITIZED SQL (FOR EXECUTION):\n', sql_to_run, '\n')

SANITIZED SQL (FOR EXECUTION):
 SELECT TOP (10)
    LoanId,
    LoanNumber,
    CompanyName,
    Industry,
    CreditRating,
    CountryName,
    RegionName,
    OriginationDate,
    MaturityDate,
    PrincipalAmount,
    CurrencyCode,
    InterestRatePct,
    InterestRateType,
    ReferenceRate,
    SpreadBps,
    AmortizationType,
    PaymentFreqMonths,
    Status,
    Purpose
FROM dbo.vw_LoanPortfolio
ORDER BY OriginationDate DESC, LoanId DESC 



## Execution (diagram: EXECUTION)
Run the SQL against Azure SQL and format results (toggle NO_EXEC for dry runs).

In [ ]:
from pathlib import Path

def _add_repo_root_to_sys_path_for_exec(target_files=("sql_executor.py",)) -> Path | None:
    cwd = Path(os.getcwd()).resolve()
    for parent in [cwd] + list(cwd.parents)[:5]:
        for fname in target_files:
            if (parent / fname).exists():
                parent_str = str(parent)
                if parent_str not in sys.path:
                    sys.path.insert(0, parent_str)
                return parent
    return None

def _safe_import_sql_executor():
    _add_repo_root_to_sys_path_for_exec(("sql_executor.py", "nl2sql_main.py"))
    try:
        from sql_executor import execute_sql_query  # type: ignore
        return execute_sql_query
    except Exception as e:
        raise ImportError('Unable to import execute_sql_query from sql_executor.') from e

def _format_table(rows):
    if not rows:
        return 'No results returned.\n', []
    columns = list(rows[0].keys())
    col_widths = {c: max(len(c), max(len(str(r[c])) for r in rows)) for c in columns}
    header = ' | '.join(c.ljust(col_widths[c]) for c in columns)
    sep = '-+-'.join('-' * col_widths[c] for c in columns)
    lines = [header, sep]
    for r in rows:
        lines.append(' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns))
    return (
        '\n'.join(lines) + '\n',
        [header, sep] + [' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns) for r in rows]
    )

# Note: Execution is controlled by the global NO_EXEC flag in the Demo settings cell.
# This cell only defines helpers; the end-to-end runner performs the actual execution.

[INFO] Execution skipped (NO_EXEC=True)


## End-to-end runner cell (adjust toggles)
Change `test_query`, `REFRESH_SCHEMA`, `SHOW_REASONING`, and `NO_EXEC` above and re-run cells.
This cell illustrates the typical order: Input → Intent → (Reasoning) → SQL → Sanitize → (Execute).

In [9]:
# Example: run the entire flow again with a different query (hardened)
test_query = 'For each region, list the top 5 companies by balance'
intent_entities = parse_nl_query(test_query)
print('INTENT & ENTITIES:\n', intent_entities, '\n')

if SHOW_REASONING:
    try:
        r = generate_reasoning(intent_entities)
        print('SQL GENERATION REASONING:\n' + (r if (r and r.strip()) else '[no reasoning returned]') + '\n')
    except Exception as e:
        print('[WARN] Reasoning step failed:', e)

# SQL generation with guard
try:
    raw_sql = generate_sql(intent_entities)
except Exception as e:
    raw_sql = ''
    print('[ERROR] SQL generation failed:', e)

if not raw_sql or not raw_sql.strip():
    print('GENERATED SQL (RAW): [empty]')
    sql_to_run = ''
    print('SANITIZED SQL (FOR EXECUTION): [empty]')
else:
    print('GENERATED SQL (RAW):\n', raw_sql, '\n')
    sql_to_run = extract_and_sanitize_sql(raw_sql)
    print('SANITIZED SQL (FOR EXECUTION):\n', sql_to_run, '\n')

# Execution with safety checks
if not NO_EXEC:
    sql_final = sql_to_run if 'sql_to_run' in locals() else ''
    if not sql_final or not sql_final.strip():
        print('[INFO] Skipping execution: SQL is empty.')
    elif not re.search(r'\bselect\b', sql_final, re.IGNORECASE):
        print('[INFO] Skipping execution: No SELECT statement detected.')
    else:
        try:
            execute_sql_query = _safe_import_sql_executor()
            rows = execute_sql_query(sql_final)
            table_text, _ = _format_table(rows)
            print('SQL QUERY RESULTS (TABLE):\n' + table_text)
        except Exception as e:
            print('[ERROR] Failed to execute SQL query:', e)
else:
    print('[INFO] Execution skipped (NO_EXEC=True)')

INTENT & ENTITIES:
 {
  "intent": "Retrieve the top 5 companies by balance within each region",
  "entities": {
    "group_by": ["region"],
    "subject": "companies",
    "metric": "balance",
    "rank": {
      "k": 5,
      "order": "desc",
      "per_group": true
    }
  },
  "expected_output_fields": ["region", "company", "balance"],
  "filters": []
} 

SQL GENERATION REASONING:
- Entities mapping: Region → vw_LoanPortfolio.RegionName; Company → vw_LoanPortfolio.CompanyName; Balance → SUM of vw_LoanPortfolio.PrincipalAmount.
- Tables/Joins: Use dbo.vw_LoanPortfolio as the sole source (denormalized). No joins needed unless currency normalization is required (not requested).
- Aggregations: SUM(PrincipalAmount) grouped by RegionName and CompanyName to compute company balance per region.
- Ranking: Use a window function (e.g., ROW_NUMBER) partitioned by RegionName, ordered by aggregated Balance DESC to get top 5 per region.
- Filters: None specified; optionally exclude NULL RegionNam

# NL→SQL Demo Notebook (Azure OpenAI + Azure SQL)

This notebook demonstrates the NL→Intent→SQL pipeline step by step, aligned with the project's flowchart.
Each step includes explanation cells for demo purposes (e.g., for a TERADATA partner presentation).

Prereqs: ensure you have a valid .env configured for Azure OpenAI and Azure SQL (see README).

In [10]:
# 1) Imports and environment bootstrap
import os, re, sys
from typing import List, Dict, Any
from dotenv import load_dotenv
load_dotenv()
print('Environment loaded. Project root:', os.getcwd())

Environment loaded. Project root: /Users/arturoquiroga/GITHUB/AQ-NEW-NL2SQL/docs


## Azure OpenAI setup and model routing
We detect reasoning-style deployments (o-series/GPT-5) and use direct REST Chat Completions without unsupported params.
For other models, we use the LangChain AzureChatOpenAI wrapper.

In [11]:
# 2) Azure OpenAI configuration & helpers
import json, requests
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2025-04-01-preview')

def _is_reasoning_like_model(deployment_name: str | None) -> bool:
    name = (deployment_name or '').lower()
    return name.startswith('o') or name.startswith('gpt-5')

USING_REASONING = _is_reasoning_like_model(DEPLOYMENT_NAME)
print('Reasoning-style deployment:', USING_REASONING, '| Deployment:', DEPLOYMENT_NAME)

def _azure_chat_completions(messages: List[dict], max_completion_tokens: int | None = None) -> str:
    url = f"{ENDPOINT.rstrip('/')}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    payload: Dict[str, Any] = {'messages': messages}
    if max_completion_tokens is not None:
        payload['max_completion_tokens'] = max_completion_tokens
    headers = {'api-key': API_KEY or '', 'Content-Type': 'application/json'}
    resp = requests.post(url, headers=headers, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    return data['choices'][0]['message']['content']

def _make_llm():
    if USING_REASONING:
        return None
    return AzureChatOpenAI(
        openai_api_key=API_KEY,
        azure_endpoint=ENDPOINT,
        deployment_name=DEPLOYMENT_NAME,
        api_version=API_VERSION,
        max_tokens=8192,
    )

llm = _make_llm()
print('LangChain LLM initialized:', llm is not None)

Reasoning-style deployment: True | Deployment: gpt-5
LangChain LLM initialized: False


## Prompts & function: Parse NL into intent/entities
We keep prompt strings inline to mirror the main script.

In [12]:
# 3) Prompts and parsing function
INTENT_PROMPT_TEXT = (
    "You are an expert in translating natural language to database queries.\n"
    "Extract the intent and entities from the following user input:\n"
    "{input}"
)
SQL_PROMPT_TEXT = (
    "You are an expert in SQL and Azure SQL Database. Given the following database schema and the intent/entities,\n"
    "generate a valid T-SQL query for querying the database.\n\n"
    "IMPORTANT:\n"
    "- Do NOT use USE statements (USE [database] is not supported in Azure SQL Database)\n"
    "- Generate only the SELECT query without USE or GO statements\n"
    "- Return only executable T-SQL code without markdown formatting\n"
    "- The database connection is already established to the correct database\n\n"
    "Schema:\n"
    "{schema}\n"
    "Intent and Entities:\n"
    "{intent_entities}\n\n"
    "Generate a T-SQL query that can be executed directly:\n"
)
REASONING_PROMPT_TEXT = (
    "Before writing SQL, produce a short, high-level reasoning summary for how you will construct the query,\n"
    "based on the schema and the intent/entities.\n\n"
    "Rules:\n"
    "- Do NOT include any SQL code.\n"
    "- Keep it concise (<= 150 words) and actionable.\n"
    "- Use simple bullets covering: Entities mapping, Tables/Joins, Aggregations (if any), Filters, Order/Limit, Assumptions.\n\n"
    "Schema:\n"
    "{schema}\n"
    "Intent and Entities:\n"
    "{intent_entities}\n\n"
    "Provide the reasoning summary now:\n"
)

intent_prompt = ChatPromptTemplate.from_template(INTENT_PROMPT_TEXT)
sql_prompt = ChatPromptTemplate.from_template(SQL_PROMPT_TEXT)
reasoning_prompt = ChatPromptTemplate.from_template(REASONING_PROMPT_TEXT)

def parse_nl_query(user_input: str) -> str:
    if USING_REASONING:
        prompt_text = INTENT_PROMPT_TEXT.format(input=user_input)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = intent_prompt | llm
    res = chain.invoke({'input': user_input})
    return res.content

print('parse_nl_query ready.')

parse_nl_query ready.


## Schema context (with optional refresh)
We use the local cache-backed context from `schema_reader.get_sql_database_schema_context`.

In [13]:
# 4) Schema context helpers
def _safe_import_schema_reader():
    # add project root to path for relative imports while running from docs/
    repo_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'docs' else os.getcwd()
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    try:
        from schema_reader import get_sql_database_schema_context, refresh_schema_cache  # type: ignore
        return get_sql_database_schema_context, refresh_schema_cache
    except Exception as e:
        raise ImportError('Unable to import schema helpers from schema_reader.py') from e

REFRESH_SCHEMA = True  # toggle for demo
get_schema_ctx, refresh_cache = _safe_import_schema_reader()
if REFRESH_SCHEMA:
    try:
        path = refresh_cache()
        print('[INFO] Schema cache refreshed at', path)
    except Exception as e:
        print('[WARN] Schema refresh failed:', e)

schema_preview = get_schema_ctx()
print(schema_preview.split('\n')[0])  # first line only

[INFO] Schema cache refreshed at /Users/arturoquiroga/GITHUB/AQ-NEW-NL2SQL/DATABASE_SETUP/schema_cache.json
DATABASE: CONTOSO-FI (Azure SQL)


## Optional reasoning step
Ask the model to outline a brief plan before SQL generation. Useful for transparency in demos.

In [14]:
# 5) Reasoning function

def generate_reasoning(intent_entities: str) -> str:
    schema = get_schema_ctx()
    if USING_REASONING:
        prompt_text = REASONING_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = reasoning_prompt | llm
    res = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return res.content

SHOW_REASONING = True  # toggle for demo
test_query = 'Show the 10 most recent loans'
intent_entities = parse_nl_query(test_query)
print('INTENT ENTITIES:', intent_entities[:200] + ('...' if len(intent_entities)>200 else ''))
if SHOW_REASONING:
    print('REASONING:\n', generate_reasoning(intent_entities))

INTENT ENTITIES: {
  "intent": "get_loans",
  "entities": {
    "object": "loans",
    "limit": 10,
    "order_by": {
      "field": "created_at",
      "direction": "desc"
    }
  }
}
REASONING:
 - Entities mapping: “loans” → rows in dbo.vw_LoanPortfolio.
- Tables/Joins: Use dbo.vw_LoanPortfolio only (denormalized; no joins needed).
- Aggregations: None (simple list).
- Filters: None specified; include all loans regardless of status.
- Order/Limit: Sort by OriginationDate DESC (mapping “created_at” → OriginationDate); return TOP 10.
- Columns to select: LoanId, LoanNumber, CompanyName, OriginationDate, PrincipalAmount, CurrencyCode, Status (and optionally Industry/Purpose if helpful).
- Assumptions: There is no created_at column; use OriginationDate as the closest proxy. If a true system “created_at” exists elsewhere, replace the sort field with that.


## SQL generation
Generate T-SQL for the given intent/entities and schema context.

In [15]:
# 6) SQL generation function

def generate_sql(intent_entities: str) -> str:
    schema = get_schema_ctx()
    if USING_REASONING:
        prompt_text = SQL_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = sql_prompt | llm
    result = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return result.content

raw_sql = generate_sql(intent_entities)
print('RAW SQL (truncated):\n', (raw_sql[:500] + '...') if len(raw_sql)>500 else raw_sql)

RAW SQL (truncated):
 SELECT TOP (10)
    LoanId,
    LoanNumber,
    CompanyName,
    Industry,
    CreditRating,
    CountryName,
    RegionName,
    OriginationDate,
    MaturityDate,
    PrincipalAmount,
    CurrencyCode,
    InterestRatePct,
    InterestRateType,
    ReferenceRate,
    SpreadBps,
    AmortizationType,
    PaymentFreqMonths,
    Status,
    Purpose
FROM dbo.vw_LoanPortfolio
ORDER BY OriginationDate DESC


## SQL sanitization
Extract only the executable SQL portion and normalize quotes for safety.

In [16]:
# 7) SQL sanitization
def extract_and_sanitize_sql(raw_sql: str) -> str:
    sql_code = raw_sql
    m = re.search(r"```sql\s*([\s\S]+?)```", raw_sql, re.IGNORECASE)
    if not m:
        m = re.search(r"```([\s\S]+?)```", raw_sql)
    if m:
        sql_code = m.group(1).strip()
    else:
        m2 = re.search(r"(SELECT[\s\S]+)", raw_sql, re.IGNORECASE)
        if m2:
            sql_code = m2.group(1).strip()
    return (sql_code
            .replace('’', "'")
            .replace('‘', "'")
            .replace('“', '"')
            .replace('”', '"'))

sql_to_run = extract_and_sanitize_sql(raw_sql)
print('SANITIZED SQL:\n', sql_to_run)

SANITIZED SQL:
 SELECT TOP (10)
    LoanId,
    LoanNumber,
    CompanyName,
    Industry,
    CreditRating,
    CountryName,
    RegionName,
    OriginationDate,
    MaturityDate,
    PrincipalAmount,
    CurrencyCode,
    InterestRatePct,
    InterestRateType,
    ReferenceRate,
    SpreadBps,
    AmortizationType,
    PaymentFreqMonths,
    Status,
    Purpose
FROM dbo.vw_LoanPortfolio
ORDER BY OriginationDate DESC


## Execution (optional)
Use `sql_executor.execute_sql_query` to run against Azure SQL. Keep NO_EXEC=True for dry runs during demos.

In [17]:
# 8) SQL execution helpers

def _safe_import_sql_executor():
    repo_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'docs' else os.getcwd()
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    try:
        from sql_executor import execute_sql_query  # type: ignore
        return execute_sql_query
    except Exception as e:
        raise ImportError('Unable to import execute_sql_query from sql_executor.py') from e


def _format_table(rows):
    if not rows:
        return 'No results returned.\n', []
    columns = list(rows[0].keys())
    col_widths = {c: max(len(c), max(len(str(r[c])) for r in rows)) for c in columns}
    header = ' | '.join(c.ljust(col_widths[c]) for c in columns)
    sep = '-+-'.join('-' * col_widths[c] for c in columns)
    lines = [header, sep]
    for r in rows:
        lines.append(' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns))
    return ('\n'.join(lines) + '\n', [header, sep] + [' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns) for r in rows])

# Default to no execution here; the end-to-end runner cell controls execution.
NO_EXEC = True  # default; override in the runner cell if you want to execute

## End-to-end demo cell
Update toggles (`REFRESH_SCHEMA`, `SHOW_REASONING`, `NO_EXEC`) and `test_query` above as needed.
Run this cell to repeat the flow with a different question.

In [ ]:
# 9) Rerun flow with another question (single, hardened output)

# Pick your test query (uses global default unless you set a new one here)
user_query = default_test_query

# 1) Intent/entities
intent_entities = parse_nl_query(user_query)
if SHOW_INTENT:
    print('INTENT ENTITIES:', intent_entities[:200] + ('...' if len(intent_entities)>200 else ''))

# 2) Optional reasoning
if SHOW_REASONING:
    try:
        r = generate_reasoning(intent_entities)
        print('REASONING:\n' + (r if (r and r.strip()) else '[no reasoning returned]'))
    except Exception as e:
        print('[WARN] Reasoning step failed:', e)

# 3) SQL generation with guard
try:
    raw_sql = generate_sql(intent_entities)
except Exception as e:
    raw_sql = ''
    print('[ERROR] SQL generation failed:', e)

if not raw_sql or not raw_sql.strip():
    print('RAW SQL (truncated): [empty]')
    sql_to_run = ''
    print('SANITIZED SQL: [empty]')
else:
    print('RAW SQL (truncated):\n', (raw_sql[:500] + '...') if len(raw_sql)>500 else raw_sql)
    sql_to_run = extract_and_sanitize_sql(raw_sql)
    print('SANITIZED SQL:\n', sql_to_run)

# 4) Execution with safety checks
if not NO_EXEC:
    sql_final = sql_to_run if 'sql_to_run' in locals() else ''
    if not sql_final or not sql_final.strip():
        print('[INFO] Skipping execution: SQL is empty.')
    elif not re.search(r'\bselect\b', sql_final, re.IGNORECASE):
        print('[INFO] Skipping execution: No SELECT statement detected.')
    else:
        try:
            execute_sql_query = _safe_import_sql_executor()
            rows = execute_sql_query(sql_final)
            table_text, _ = _format_table(rows)
            print('RESULTS:\n' + table_text)
        except Exception as e:
            print('[ERROR] Query failed:', e)
else:
    print('[INFO] Execution skipped (NO_EXEC=True)')

INTENT ENTITIES: {
  "intent": "compute_weighted_average_by_group",
  "entities": {
    "aggregation": "weighted_average",
    "measure": "interest_rate",
    "weight": null,
    "group_by": ["region"],
    "filters":...
REASONING:
- Entities mapping: interest_rate -> vw_LoanPortfolio.InterestRatePct; region -> vw_LoanPortfolio.RegionName; weight -> PrincipalAmount (portfolio guideline).
- Tables/Joins: Use dbo.vw_LoanPortfolio only (no join needed).
- Aggregations: Weighted average by region = sum(InterestRatePct * PrincipalAmount) / sum(PrincipalAmount), grouped by RegionName.
- Filters: Exclude rows with null InterestRatePct or null/zero PrincipalAmount; no time filter applied; optionally include only loans with non-cancelled Status if desired.
- Order/Limit: Order by RegionName ascending; no limit.
- Assumptions: 
  - Weight = PrincipalAmount (since balance not provided).
  - Region field is RegionName in vw_LoanPortfolio.
  - Include all loans regardless of origination/maturity un